In [1]:
import pandas as pd

In [2]:
prediction_data = pd.read_csv('Results.csv', index_col=0)

In [3]:
dates = pd.read_csv('Dates.csv', index_col=0)

In [4]:
news_data = pd.read_csv('News.csv')

In [5]:
news_data

,ticker,date,headline
0,AAPL,2021-07-28 07:55:00,"Dow Jones Futures: Apple, Microsoft, Google, A..."
1,AAPL,2021-07-28 07:17:00,The real opportunities are with Empire Strikes...
2,AAPL,2021-07-28 06:30:00,Apple Stock Is Under Pressure. Theres a Silver...
3,AAPL,2021-07-28 06:10:00,Apple Earnings Were Out of This World. Why the...
4,AAPL,2021-07-28 06:03:00,3 High-Growth Stocks That Could Be Worth $1 Tr...
...,...,...,...
95,AAPL,2021-07-25 19:16:00,"The main attractions arrive: Apple, Microsoft,..."
96,AAPL,2021-07-25 11:43:00,"Big Tech earnings, Federal Reserve decision: W..."
97,AAPL,2021-07-25 07:00:00,3 Things to Watch in the Stock Market This Week
98,AAPL,2021-07-25 06:10:00,3 Hot Tech Stocks to Buy in a Market Crash


In [6]:
#prediction_data.drop(prediction_data.index[:1], inplace=True)

prediction_data = prediction_data.iloc[1:]


In [7]:
prediction_data['date'] = news_data['date']




In [8]:
prediction_data.dropna(axis=0, inplace=True)
prediction_data.set_index(['date'], inplace=True)
df = prediction_data

In [9]:
dates = pd.to_datetime(prediction_data.index)
df1 = pd.DataFrame(prediction_data['sentiment_score'], index = dates)
df1 = df1.iloc[::-1]


In [10]:
#df1.Date=pd.to_datetime(df1['date'])
#df1['date'] = df1['date'].apply(lambda x: x.strftime('%Y-%m-%d %H: %M: %S'))


In [24]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('exitbars', 0),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datasent = self.datas[0].sent

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.datasent[0] >0.5:
              # BUY, BUY, BUY!!! (with default parameters)
              self.log('BUY CREATE, %.2f' % self.dataclose[0])

              # Keep track of the created order to avoid a 2nd order
              self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if self.datasent[0] < -0.5:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
#!pip install yfinance
#!pip install mplfinance
import yfinance as yf
#import mplfinance as mpf
from datetime import datetime
sd = datetime(2020, 8, 24)
ed = datetime(2020, 8, 25)
df = yf.download(tickers='AMZN', start=sd, end=ed, interval="1m")
#mpf.plot(df,type='candle',mav=(3,6,9),volume=True)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/yfinance/multi.py", line 167, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "/opt/anaconda3/lib/python3.7/site-packages/yfinance/multi.py", line 182, in _download_one
    rounding=rounding, many=True)
  File "/opt/anaconda3/lib/python3.7/site-packages/yfinance/base.py", line 156, in history
    data = data.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 898, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python

In [25]:
from datetime import datetime, timezone
import datetime as dt
yf = pd.read_csv('AAPL.csv')
yf.rename(columns={'Local time':'Date'}, inplace=True)
yf.Date=pd.to_datetime(yf['Date'])
yf['Date'] = yf['Date'].astype('datetime64[ns]')
yf.set_index(['Date'], inplace=True)



,Open,High,Low,Close,Volume,sent
2021-07-23 02:00:00,146.803,146.803,146.803,146.803,0.0,0.0
2021-07-23 02:01:00,146.803,146.803,146.803,146.803,0.0,0.0
2021-07-23 02:02:00,146.803,146.803,146.803,146.803,0.0,0.0


In [26]:
comb=yf.merge(df1,left_index=True,right_index=True,how='left').fillna(0)
comb = pd.DataFrame(comb)
comb.rename(columns={'sentiment_score':'sent'}, inplace=True)
comb.head(3)

In [27]:
class PandasSent(bt.feeds.PandasData):
    lines=(('sent'),)
    params=(('sent',-1),)

In [28]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    data = PandasSent(dataname=comb)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)

    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
20

2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, Close, 146.80
2021-07-23, 

2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, 

2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, 

2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, Close, 148.57
2021-07-24, 

...

2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.57
2021-07-26, Close, 148.53
2021-07-26, Close, 148.10
2021-07-26, Close, 148.16
2021-07-26, Close, 147.80
2021-07-26, Close, 147.89
2021-07-26, Close, 148.13
2021-07-26, Close, 148.29
2021-07-26, Close, 148.53
2021-07-26, Close, 148.59
2021-07-26, Close, 148.84
2021-07-26, Close, 149.04
2021-07-26, Close, 148.88
2021-07-26, Close, 148.91
2021-07-26, Close, 148.97
2021-07-26, Close, 149.24
2021-07-26, Close, 149.38
2021-07-26, Close, 149.66
2021-07-26, Close, 149.35
2021-07-26, Close, 149.41
2021-07-26, Close, 149.54
2021-07-26, Close, 149.50
2021-07-26, Close, 149.62
2021-07-26, 

2021-07-26, Close, 149.41
2021-07-26, Close, 149.45
2021-07-26, Close, 149.46
2021-07-26, Close, 149.40
2021-07-26, Close, 149.38
2021-07-26, Close, 149.39
2021-07-26, Close, 149.35
2021-07-26, Close, 149.29
2021-07-26, Close, 149.30
2021-07-26, Close, 149.26
2021-07-26, Close, 149.25
2021-07-26, Close, 149.26
2021-07-26, Close, 149.25
2021-07-26, Close, 149.27
2021-07-26, Close, 149.22
2021-07-26, Close, 149.21
2021-07-26, Close, 149.12
2021-07-26, Close, 149.10
2021-07-26, Close, 149.10
2021-07-26, Close, 149.01
2021-07-26, Close, 148.97
2021-07-26, Close, 148.92
2021-07-26, Close, 148.95
2021-07-26, Close, 148.88
2021-07-26, Close, 148.81
2021-07-26, Close, 148.81
2021-07-26, Close, 148.93
2021-07-26, Close, 148.97
2021-07-26, Close, 149.00
2021-07-26, Close, 148.93
2021-07-26, Close, 148.94
2021-07-26, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, 

2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, 

2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, Close, 148.99
2021-07-27, 

2021-07-27, BUY EXECUTED, Price: 147.83, Cost: 14783.30, Comm 14.78
2021-07-27, Close, 147.71
2021-07-27, Close, 147.50
2021-07-27, Close, 147.17
2021-07-27, Close, 146.98
2021-07-27, Close, 147.14
2021-07-27, Close, 147.12
2021-07-27, Close, 147.33
2021-07-27, Close, 147.31
2021-07-27, Close, 147.59
2021-07-27, Close, 147.60
2021-07-27, Close, 147.67
2021-07-27, Close, 147.58
2021-07-27, Close, 147.49
2021-07-27, Close, 147.48
2021-07-27, Close, 147.55
2021-07-27, Close, 147.44
2021-07-27, Close, 147.39
2021-07-27, Close, 147.21
2021-07-27, Close, 147.08
2021-07-27, Close, 147.09
2021-07-27, Close, 146.82
2021-07-27, Close, 146.85
2021-07-27, Close, 147.01
2021-07-27, Close, 146.66
2021-07-27, Close, 146.77
2021-07-27, Close, 146.78
2021-07-27, Close, 146.78
2021-07-27, Close, 146.85
2021-07-27, Close, 146.82
2021-07-27, Close, 147.00
2021-07-27, Close, 147.01
2021-07-27, Close, 147.15
2021-07-27, Close, 147.10
2021-07-27, Close, 147.17
2021-07-27, Close, 147.17
2021-07-27, Close, 147